In [2]:
#Instalando pacotes
!pip install sweetviz
!pip install pygwalker
!conda install -c conda-forge pygwalker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 29.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.2 MB/s eta 0:00:00
/bin/bash: conda: command not found


In [3]:
#Importa as bibliotecas para o projeto
import pandas as pd
import numpy as np
import sweetviz as sv
import pygwalker as pyg
import plotly.express as px
from datetime import datetime
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive


In [4]:
#Monta o drive e coleta as credênciais
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Mounted at /content/drive


In [5]:

#cria variável para receber o arquivo CSV via 'id'
downloaded = drive.CreateFile({'id':'1SxnX-rxWyum-HyOyjx5OAgRy0hCucU0r'})

content = downloaded.GetContentString()
#https://drive.google.com/file/d/1SxnX-rxWyum-HyOyjx5OAgRy0hCucU0r/view?usp=share_link


In [6]:
#Gera um dataframe pandas separando as colunas por ','
df = pd.DataFrame([row.split(',') for row in content.split('\n')])

#Transforma a primeira linha do dataframe em colunas
df.columns = df.iloc[0]
df = df[1:]

In [7]:
#Verifica o tipo dos dados
df.dtypes

0
#              object
department     object
job_title      object
send_at        object
survey         object
user_id        object
answer         object
received_at    object
dtype: object

# Tratamento dos dados referente às respostas

In [8]:
#transforma o tipo dos dados para o formato de data
df['send_at'] = pd.to_datetime(df['send_at'])
df['received_at'] = pd.to_datetime(df['received_at'])
df.dtypes


0
#                           object
department                  object
job_title                   object
send_at        datetime64[ns, UTC]
survey                      object
user_id                     object
answer                      object
received_at    datetime64[ns, UTC]
dtype: object

In [9]:
# Adicionando uma coluna com o número da semana do mês
df['week'] = df['send_at'].apply(lambda x: (x.day - 1) // 7 + 1)


In [117]:
df['response_day_diff'] = df['received_at'] - df['send_at']

In [11]:
#Cria-se uma coluna para receber a diferença de tempo entre o envio e a resposta da pesquisa
df.response_day_diff = df.response_day_diff.dt.days
df.head()

,#,department,job_title,send_at,survey,user_id,answer,received_at,week,response_day_diff
1,1,Sucesso do Cliente,CS,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100089455296709,,NaT,3.0,NaN
2,2,Marketing,Marketing,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100089043811982,ESTRESSADO,2023-01-20 18:04:26+00:00,3.0,0.0
3,3,Tecnologia,Desenvolvedor,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100088080756062,FELIZ,2023-01-20 18:18:35+00:00,3.0,0.0
4,4,Operações,BDR,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100087409901143,EMPOLGADO,2023-01-20 18:01:00+00:00,3.0,0.0
5,5,Tecnologia,Desenvolvedor,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100087321184130,EMPOLGADO,2023-01-20 19:03:34+00:00,3.0,0.0


In [12]:
df.dtypes


0
#                                 object
department                        object
job_title                         object
send_at              datetime64[ns, UTC]
survey                            object
user_id                           object
answer                            object
received_at          datetime64[ns, UTC]
week                             float64
response_day_diff                float64
dtype: object

In [ ]:
#Gera uma análise exploratória inicial
my_report = sv.analyze(df)
my_report.show_notebook()

In [14]:
#preenche os registros sem resposta como "-1.0"
df = df.fillna({'response_day_diff':float(-1.0)})
df.dtypes


0
#                                 object
department                        object
job_title                         object
send_at              datetime64[ns, UTC]
survey                            object
user_id                           object
answer                            object
received_at          datetime64[ns, UTC]
week                             float64
response_day_diff                float64
dtype: object

In [15]:
#função que recebe uma coluna e retorna a coluna com as respostas válidas
def mesmo_dia(column):
      
      if column >= 1.0:  
          column = "invalid"

      elif column == 0.0:        
           column = "valid"
      
      elif column == -1.0:
           column = "no_response"
           
      return str(column)

In [16]:
# Adiciona uma coluna ao dataset com as validade das respostas
df['response'] = df['response_day_diff'].apply(mesmo_dia)

In [17]:
df.head()

,#,department,job_title,send_at,survey,user_id,answer,received_at,week,response_day_diff,response
1,1,Sucesso do Cliente,CS,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100089455296709,,NaT,3.0,-1.0,no_response
2,2,Marketing,Marketing,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100089043811982,ESTRESSADO,2023-01-20 18:04:26+00:00,3.0,0.0,valid
3,3,Tecnologia,Desenvolvedor,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100088080756062,FELIZ,2023-01-20 18:18:35+00:00,3.0,0.0,valid
4,4,Operações,BDR,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100087409901143,EMPOLGADO,2023-01-20 18:01:00+00:00,3.0,0.0,valid
5,5,Tecnologia,Desenvolvedor,2023-01-20 18:00:00.008000+00:00,FELICIDADE,100087321184130,EMPOLGADO,2023-01-20 19:03:34+00:00,3.0,0.0,valid


In [38]:
df_response = df.response.value_counts()
df_response = pd.DataFrame(df_response)
df_response

,response
valid,1163
no_response,433
invalid,21


In [59]:
df_valid_responses =  df[ df['response'] != 'invalid']
df_valid_responses.dropna(inplace = True)



/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


# Análise de sentimentos por semana do mês

In [60]:
df_valid_responses.answer.value_counts()

TRANQUILO            287
FELIZ                233
EMPOLGADO            162
CANSADO              160
ANSIOSO               99
COM_MUITA_ENERGIA     57
DOENTE                49
ESTRESSADO            42
DESMOTIVADO           41
TRISTE                30
BEM                    3
Name: answer, dtype: int64

In [129]:
df_responses_feeling = df_valid_responses.pivot_table(index = 'week', columns = ['answer'], values= "#", aggfunc = 'count').reset_index()
df_responses_feeling


answer,week,ANSIOSO,BEM,CANSADO,COM_MUITA_ENERGIA,DESMOTIVADO,DOENTE,EMPOLGADO,ESTRESSADO,FELIZ,TRANQUILO,TRISTE
0,1.0,26.0,NaN,32.0,9.0,11.0,13.0,36.0,10.0,44.0,67.0,4.0
1,2.0,24.0,NaN,38.0,15.0,5.0,10.0,32.0,16.0,41.0,49.0,8.0
2,3.0,11.0,NaN,28.0,7.0,6.0,4.0,33.0,3.0,49.0,44.0,2.0
3,4.0,34.0,3.0,49.0,18.0,16.0,19.0,50.0,10.0,79.0,92.0,12.0
4,5.0,4.0,NaN,13.0,8.0,3.0,3.0,11.0,3.0,20.0,35.0,4.0


In [119]:
df_responses_tired_rate = pd.DataFrame(data = None)
df_responses_tired_rate['week'] = df_responses_feeling.groupby(['week']).count().index
df_responses_tired_rate['tired'] = df_responses_feeling['CANSADO'].values
df_responses_tired_rate['Prop(%)'] = round(df_responses_feeling['CANSADO']/sum(df_responses_feeling['CANSADO'])*100,2)
df_responses_tired_rate

,week,tired,Prop(%)
0,1.0,32.0,20.00
1,2.0,38.0,23.75
2,3.0,28.0,17.50
3,4.0,49.0,30.63
4,5.0,13.0,8.12


In [110]:
df_responses_anxiety_rate = pd.DataFrame(data = None)
df_responses_anxiety_rate['week'] = df_responses_feeling.groupby(['week']).count().index
df_responses_anxiety_rate['anxiety'] = df_responses_feeling['ANSIOSO'].values
df_responses_anxiety_rate['Prop(%)'] = round(df_responses_feeling['ANSIOSO']/sum(df_responses_feeling['ANSIOSO'])*100,2)
df_responses_anxiety_rate

,week,anxiety,Prop(%)
0,1.0,26.0,26.26
1,2.0,24.0,24.24
2,3.0,11.0,11.11
3,4.0,34.0,34.34
4,5.0,4.0,4.04


In [ ]:
fig_anxiety = px.bar(df_responses_anxiety_rate , x='week', y= 'anxiety', color = 'week' , title = 'Ansiedade por semana do Mês')
fig_anxiety.show()

fig_tired = px.bar(df_responses_tired_rate , x='week', y= 'tired', color = 'week', title = 'Cansaço por semana do Mês')
fig_tired.show()

# Análise do engajamento por colaborador 

In [220]:
df_responses_users = df_valid_responses.pivot_table(index = ['user_id', 'job_title'], values= 'answer', aggfunc = 'count').reset_index()
df_responses_users = df_responses_users.sort_values(by = 'answer', ascending = False)
df_responses_users.fillna(0, inplace = True)
df_responses_users

,user_id,job_title,answer
4,100080263284452,Liderança,25
15,100080459655282,Finder,25
32,100080680327605,Vendedor,24
51,100084144069825,BDR,24
1,100080182879558,Gestão,24
...,...,...,...
54,100084314808556,BDR,4
19,100080493613478,Vendedor,1
42,100081249500240,Marketing,1
63,100089461190980,BDR,1


In [221]:
df_responses_users.groupby('job_title').mean().reset_index().sort_values(by = 'answer', ascending = False)

,job_title,answer
1,C-Level,23.333333
12,Tech,22.666667
6,Finder,21.666667
5,Financeiro,21.000000
10,RH,21.000000
2,CS,19.400000
4,Desenvolvedor,19.000000
8,Liderança,18.714286
7,Gestão,18.000000
0,BDR,17.210526
